In [1]:
import pandas as pd
import numpy as np
import yaml

try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df = pd.read_csv(config['output_data']['file'])

In [2]:
df

,age,gender,weight_kg,height_m,max_bpm,avg_bpm,resting_bpm,session_duration_hours,calories_burned,workout_type,...,carbs,proteins,fats,calories,meal_type,diet_type,pct_carbs,protein_per_kg,cal_balance,lean_mass_kg
0,34,Male,65,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,...,267.68,106.05,71.63,1806.0,Lunch,Vegan,0.500432,1.624789,725.10,47.777394
1,23,Female,56,1.55,179.43,131.75,73.18,1.37,1809.91,Hiit,...,214.32,85.41,56.97,1577.0,Lunch,Vegetarian,0.500850,1.514093,-232.91,40.809803
2,33,Female,58,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,...,246.04,98.11,65.48,1608.0,Breakfast,Paleo,0.500610,1.663445,805.74,44.635580
3,38,Female,93,1.70,191.21,155.10,50.07,1.10,1450.79,Hiit,...,203.22,80.84,54.56,2657.0,Lunch,Paleo,0.499533,0.862017,1206.21,63.007432
4,45,Male,52,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,...,332.79,133.05,88.43,1470.0,Breakfast,Vegan,0.500581,2.538153,303.60,43.347504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,46,Female,98,1.90,199.20,148.18,63.72,0.77,761.61,Strength,...,235.35,93.87,62.88,2627.0,Breakfast,Low-Carb,0.500000,0.954837,1865.39,71.269345
19996,40,Female,88,1.87,196.18,134.18,54.04,1.97,2303.13,Strength,...,149.77,60.72,40.39,2477.0,Snack,Vegetarian,0.496968,0.689060,173.87,65.049689
19997,50,Male,46,1.67,163.34,157.92,61.65,1.36,1468.80,Strength,...,243.61,97.29,64.91,1425.0,Snack,Keto,0.500280,2.105844,-43.80,35.420708
19998,52,Male,44,1.62,179.27,121.23,60.88,1.41,929.75,Yoga,...,240.16,97.05,63.63,1276.0,Snack,Paleo,0.499940,2.190745,346.25,35.889260


In [ ]:
def derive_body_metrics(df):
  
    age = int(input("Enter your age (years): "))
    gender = input("Enter your gender (Male/Female): ").strip().title()
    height_m = float(input("Enter your height (in m): "))
    weight_kg = int(input("Enter your weight (in kg): "))

    if not (18 <= age <= 59):
        print("Age must be between 18 and 59 years")
        return None

    if not (1.49 <= height_m <= 2.01):
        print("Height must be between 1.49 m and 2.01 m")
        return None

    if not (39 <= weight_kg <= 130):
        print("Weight must be between 39 kg and 130 kg")
        return None

#Check if record exists in df
    df_gender = df[df['gender'].str.title() == gender].copy()

    if df_gender.empty:
        print("No records found for this gender.")
        return None

    # --- Compute difference to find nearest record ---
    df_gender["age_diff"] = abs(df_gender["age"] - age)
    df_gender["height_diff"] = abs(df_gender["height_m"] - height_m)
    df_gender["weight_diff"] = abs(df_gender["weight_kg"] - weight_kg)

    # Combine differences into a single distance score
    df_gender["total_diff"] = df_gender["age_diff"] + df_gender["height_diff"] + (df_gender["weight_diff"] / 10)

    # Pick the closest record
    match = df_gender.nsmallest(1, "total_diff")

    # --- Display matched record info ---
    matched_row = match.iloc[0]
    print(f"\n Closest match found in dataset: Age {matched_row['age']}, "
          f"Height {matched_row['height_m']} m, Weight {matched_row['weight_kg']} kg")

    # Calculations
    bmi = weight_kg / (height_m ** 2)
    gender_factor = 1 if gender.lower() == "male" else 0
    body_fat_percentage = (1.20 * bmi) + (0.23 * age) - (10.8 * gender_factor) - 5.4
    lean_mass_kg = weight_kg * (1 - body_fat_percentage / 100)

    # --- BMI Category Recommendation ---
    if bmi < 18.5:
        bmi_category = "Underweight"
        recommendation = "Increase calorie intake and focus on nutrient-rich foods."
    elif 18.5 <= bmi < 25:
        bmi_category = "Normal weight"
        recommendation = "Maintain current lifestyle with balanced diet and exercise."
    elif 25 <= bmi < 30:
        bmi_category = "Overweight"
        recommendation = "Include more physical activity and moderate calorie control."
    elif 30 <= bmi < 40:
        bmi_category = "Obese"
        recommendation = "Adopt a structured diet and regular exercise plan; consider medical advice."
    else:
        bmi_category = "Severely Obese"
        recommendation = "Seek professional guidance for a supervised weight management program."

    # --- Results ---
    result = {
        "Age": age,
        "Gender": gender,
        "Height (m)": height_m,
        "Weight (kg)": weight_kg,
        "BMI": round(bmi, 2),
        "BMI Category": bmi_category,
        "Recommendation": recommendation,
        "Body Fat %": round(body_fat_percentage, 2),
        "Lean Mass (kg)": round(lean_mass_kg, 2) }
    return result

result = derive_body_metrics(df)

if result:
    print("\n=== Derived Fitness Metrics ===")
    for key, value in result.items():
        print(f"{key}: {value}")

In [31]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

ohe = OneHotEncoder(sparse_output=False, drop="first")

categorical_columns = df.select_dtypes(include="object").columns.tolist()
numerical_columns = df.select_dtypes(include="number").columns.tolist()

ohe.fit(df[categorical_columns])
categorical_trans_np = ohe.transform(df[categorical_columns])

categorical_trans_df = pd.DataFrame(categorical_trans_np, columns=ohe.get_feature_names_out(), index=df.index)

df_numerical = df.select_dtypes("number")
df_trans = pd.concat([categorical_trans_df, df_numerical], axis=1)

scaler = StandardScaler()
scaler.fit(df_trans)
df_trans_final_np = scaler.transform(df_trans)
df_trans_final = pd.DataFrame(df_trans_final_np, columns=df_trans.columns, index=df_trans.index)

nn = NearestNeighbors(n_neighbors=1)
nn.fit(df_trans_final)

values = {}
for col in df.columns:
    col_value = input(f"Please, enter your {col}: ")

    if col in numerical_columns:
        try:
            col_value = float(col_value)
        except ValueError:
            pass
    values[col] = col_value


new_input_df = pd.DataFrame([values])

categorical_trans_user = ohe.transform(new_input_df[categorical_columns])
categorical_trans_user_df = pd.DataFrame(categorical_trans_user, columns=ohe.get_feature_names_out(), index=new_input_df.index)

df_numerical_user = new_input_df[numerical_columns]
df_trans_user = pd.concat([categorical_trans_user_df, df_numerical_user], axis=1)

df_trans_final_user = pd.DataFrame(scaler.transform(df_trans_user),columns=df_trans.columns, index=new_input_df.index)


distance, index = nn.kneighbors(df_trans_final_user)
nearest_row = df.iloc[index[0][0]]


print(f"Similarity Distance: {distance[0][0]:.4f}")
print("\nMost Similar User Profile:")
print(nearest_row)

Please, enter your age:  34
Please, enter your gender:  Male
Please, enter your weight_kg:  65
Please, enter your height_m:  1.62
Please, enter your max_bpm:  181
Please, enter your avg_bpm:  157
Please, enter your resting_bpm:  69.05
Please, enter your session_duration_hours:  1
Please, enter your calories_burned:  1080
Please, enter your workout_type:  Strength
Please, enter your fat_percentage:  26.8
Please, enter your water_intake_liters:  1
Please, enter your bmi:  24.8
Please, enter your carbs:  267
Please, enter your proteins:  106
Please, enter your fats:  71
Please, enter your calories:  1806
Please, enter your meal_type:  Lunch
Please, enter your diet_type:  Vegan
Please, enter your pct_carbs:  .5
Please, enter your protein_per_kg:  1.6
Please, enter your cal_balance:  725
Please, enter your lean_mass_kg:  47.7


Similarity Distance: 0.7269

Most Similar User Profile:
age                              34
gender                         Male
weight_kg                        65
height_m                       1.62
max_bpm                      188.58
avg_bpm                      157.65
resting_bpm                   69.05
session_duration_hours          1.0
calories_burned              1080.9
workout_type               Strength
fat_percentage            26.800377
water_intake_liters               1
bmi                           24.87
carbs                        267.68
proteins                     106.05
fats                          71.63
calories                     1806.0
meal_type                     Lunch
diet_type                     Vegan
pct_carbs                  0.500432
protein_per_kg             1.624789
cal_balance                   725.1
lean_mass_kg              47.777394
Name: 0, dtype: object
